In [1]:
import qdpm
import numpy as np
import matplotlib.pyplot as plt
from qutip import identity, Qobj, tensor, basis, bell_state, ket2dm, sigmax, sigmay, sigmaz
from qutip.qip.operations import rx, ry, cphase, cnot
import itertools

In [2]:
qst_exp = qdpm.Experiment(r'D:\QPG\toTongyu\QST\YPiO2 X YPiO2\2021_02_17_19_00_41')
# qst_exp = qdpm.Experiment(r'O:\68707\BF2222\LABVIEWdata\Cooldown20210114\Heterodyne\Nb_SWIPHT_MB_W8_180nm\QST\YPiO2 X YPiO2\2021_02_17_19_00_41')

In [10]:
rotation = [identity(2),
            rx(-np.pi/2),
            ry(np.pi/2)]

initial_state = tensor((basis(2, 0) + basis(2, 1)).unit(),
                       (basis(2, 0) + basis(2, 1)).unit())
# initial_state = tensor((basis(2, 0) + basis(2, 1)).unit(),
#                        (basis(2, 0) + basis(2, 1)).unit())

In [11]:
def target_varying(phi):
    return (np.exp(-1j*phi/2)*tensor(basis(2, 1), basis(2, 0))-1j*tensor(basis(2, 0), basis(2, 1))).unit()

In [12]:
target = cphase(np.pi)*initial_state

In [13]:
fidelity = np.zeros(41)
phi = np.linspace(2*np.pi, 2.5*np.pi, 41)
for n in range(len(phi)):
    qst = qdpm.StateTomography(qst_exp, rotation, theory_state=target_varying(phi[n]))
    fidelity[n] = qst.fidelity

In [14]:
max_idx = np.argmax(fidelity)

In [15]:
qst = qdpm.StateTomography(qst_exp, rotation, theory_state=target_varying(phi[max_idx]))

In [16]:
qst.fidelity

0.43119499004164724

In [ ]:
fig, ax = plt.subplots()
qst.plot(part='real', ax=ax)
ax.set_title(r'$Re(\rho)$', fontsize=20)
# fig.savefig(qst_exp.path.joinpath('rho_MLE_real_part.png'), bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
fig, ax = plt.subplots()
qst_max.plot(part='imag', ax=ax)
ax.set_title(r'$Im(\rho)$', fontsize=20)
fig.savefig(qst_exp.path.joinpath('rho_MLE_imaginary_part.png'), bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
fig, ax = plt.subplots()
qst_max.plot_theory(part='real', ax=ax)
fig.savefig(qst_exp.path.joinpath('rho_theory_real_part.png'))
plt.show()

In [ ]:
fig, ax = plt.subplots()
qst_max.plot_theory(part='imag', ax=ax)
fig.savefig(qst_exp.path.joinpath('rho_theory_imaginary_part.png'))
plt.show()

In [ ]:
rho_mle = Qobj(qst_max.rho_mle, dims=[[2, 2], [2, 2]])

In [ ]:
pauli_1Q_rescaled = [identity(2)/np.sqrt(2),
                     sigmax()/np.sqrt(2),
                     sigmay()/np.sqrt(2),
                     sigmaz()/np.sqrt(2)]

pauli_2Q_rescaled = []
for i, j in itertools.product(range(4), repeat=2):
    pauli_2Q_rescaled.append(tensor(pauli_1Q_rescaled[i], pauli_1Q_rescaled[j]))

In [ ]:
rho_p_theory = 2*np.array([(a*ket2dm(target_varying(phi[max_idx]))).tr() for a in pauli_2Q_rescaled])
rho_p = 2*np.array([(a*rho_mle).tr() for a in pauli_2Q_rescaled])

In [ ]:
single_Q_comp = [0,1,2,3,4,8,12]
two_Q_comp = list(set(range(16))-set(single_Q_comp))

In [ ]:
fig, ax = plt.subplots(figsize=(7.8, 4.2))
ticklabel = []
for idx in itertools.product('IXYZ', repeat=2):
    ticklabel.append(''.join(map(str, idx)))
ax.set_axisbelow(True)
ax.grid(color='k', alpha=0.2, linestyle='-', linewidth=1)
ax.bar(single_Q_comp, rho_p[single_Q_comp], edgecolor='None', color=(0.6, 0.55, 1))
ax.bar(two_Q_comp, rho_p[two_Q_comp], edgecolor='None', color=(1, 0, 0))
ax.bar(range(16), rho_p_theory, edgecolor='k', color='None')
ax.set_xlim(-0.5,16-.5)
ax.set_xticks(range(16))
ax.set_xticklabels(ticklabel, fontsize=14)

fig.savefig(qst_exp.path.joinpath('Pauli_vector.png'), bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
from matplotlib.gridspec import GridSpec
import matplotlib.image as mpimg

In [ ]:
img = mpimg.imread(qst_exp.path.joinpath('scheme.png'))

In [ ]:
# def format_axes(fig):
#     for i, ax in enumerate(fig.axes):
#         ax.text(0.5, 0.5, "ax%d" % (i+1), va="center", ha="center")
#         ax.tick_params(labelbottom=False, labelleft=False)

fig = plt.figure(constrained_layout=True, figsize=(6, 8))

gs = GridSpec(3, 2, figure=fig)
ax1 = fig.add_subplot(gs[0, :])
ax2 = fig.add_subplot(gs[1, 0])
ax3 = fig.add_subplot(gs[1, 1])
ax4 = fig.add_subplot(gs[2, :])

ax1.imshow(img)
ax1.axis('off')

qst_max.plot(part='real', ax=ax2, colorbar=False)
ax2.set_title(r'Re$(\rho)$', fontsize=16)
ax2.set_xticklabels([r'$\widetilde{|00}\rightangle$', r'$\widetilde{|01}\rightangle$', r'$\widetilde{|10}\rightangle$', r'$\widetilde{|11}\rightangle$'], fontsize=14)
ax2.set_yticklabels([r'$\widetilde{|00}\rightangle$', r'$\widetilde{|01}\rightangle$', r'$\widetilde{|10}\rightangle$', r'$\widetilde{|11}\rightangle$'], fontsize=14)

_, _, img_imag = qst_max.plot(part='imag', ax=ax3, colorbar=False)
ax3.set_title(r'Im$(\rho)$', fontsize=16)
cb_ax = fig.add_axes([0.99, .41, .02, .258])
fig.colorbar(img_imag, ticks=np.linspace(-1, 1, 5), cax=cb_ax)
img_imag.set_clim(-1, 1)
ax3.set_xticklabels([r'$\widetilde{|00}\rightangle$', r'$\widetilde{|01}\rightangle$', r'$\widetilde{|10}\rightangle$', r'$\widetilde{|11}\rightangle$'], fontsize=14)
ax3.set_yticklabels([r'$\widetilde{|00}\rightangle$', r'$\widetilde{|01}\rightangle$', r'$\widetilde{|10}\rightangle$', r'$\widetilde{|11}\rightangle$'], fontsize=14)

ax4.set_axisbelow(True)
ax4.grid(color='k', alpha=0.2, linestyle='-', linewidth=1)
ax4.bar(single_Q_comp, rho_p[single_Q_comp], edgecolor='None', color=(0.6, 0.55, 1))
ax4.bar(two_Q_comp, rho_p[two_Q_comp], edgecolor='None', color=(1, 0, 0))
ax4.bar(range(16), rho_p_theory, edgecolor='k', color='None')
ax4.set_xlim(-0.5,16-.5)
ax4.set_xticks(range(16))
ax4.set_yticks(np.linspace(-1, 1, 5))
ax4.set_xticklabels(ticklabel, fontsize=14)
ax4.set_yticklabels(np.linspace(-1, 1, 5), fontsize=14)

ax1.text(-0.1, 1.1, '(a)', transform=ax1.transAxes, size=20)
ax2.text(-0.34, 1.1, '(b)', transform=ax2.transAxes, size=20)
ax3.text(-0.34, 1.1, '(c)', transform=ax3.transAxes, size=20)
ax4.text(-0.1, 1.1, '(d)', transform=ax4.transAxes, size=20)

fig.savefig(qst_exp.path.joinpath('QST.png'), bbox_inches='tight')
plt.show()

# QST Train

In [ ]:
qst_train_exp = qdpm.Experiment(r'O:\68707\JoelHoward\Experiments\EntanglementSpeedLimit\twoQubit_SWIPHT_directCapCoupling\February2021-Round4\DataAnalysis\Data\state_01+10_numSegs_4_taus_24\run0\exp_1of3')

In [ ]:
rotation = [identity(2),
            rx(-np.pi/2),
            ry(np.pi/2)]

initial_state = tensor(basis(2, 0),
                       basis(2, 0))

In [ ]:
target = (tensor(basis(2, 0), basis(2, 1))+tensor(basis(2, 1), basis(2, 0))).unit()

In [ ]:
qst = qdpm.QST_chain(qst_train_exp, rotation, theory_state=target, num_of_qubit=2, num_of_QST=11)

In [ ]:
qst.purity

In [ ]:
np.matmul(qst.rho_mle[0].conj(), qst.rho_mle[0]).trace().real

In [ ]:
np.matmul(qst.rho_mle[0], qst.rho_mle[0]).trace().real

In [ ]:
Qobj(qst.rho_mle[0])

In [ ]:
qst.plot_theory(part='imag', colorbar=True)

In [ ]:
qst.plot(part='real', QST_idx=6, colorbar=True)